In [1]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jkyeaton/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import re
import numpy as np
import pandas as pd
import pickle
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [3]:
#import pyLDAvis.gensim  # don't skip this
import os
import sys

In [4]:
#from nltk.corpus import stopwords
#stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [5]:
#parent_directory = os.path.abspath(os.path.join(os.getcwd(), '..', 'projects', 'data', 'tokenized'))
#parent_directory = os.path.abspath(os.path.join(os.getcwd(), '..'))
#print('parent', parent_directory)
#sys.path.append(parent_directory)
#print('sys', sys.path)

parent /home/jkyeaton/dsi_files_addtl/climate-conversations/project/projects/data/tokenized
sys ['/home/jkyeaton/dsi_files_addtl/climate-conversations/project/topic_modeling', '/home/jkyeaton/miniconda3/envs/climate_env/lib/python311.zip', '/home/jkyeaton/miniconda3/envs/climate_env/lib/python3.11', '/home/jkyeaton/miniconda3/envs/climate_env/lib/python3.11/lib-dynload', '', '/home/jkyeaton/miniconda3/envs/climate_env/lib/python3.11/site-packages', '/home/jkyeaton/dsi_files_addtl/climate-conversations/project/projects/data/tokenized']


In [13]:
# Your parent_directory definition
parent_directory = os.path.abspath(os.path.join(os.getcwd(), '..', 'data', 'tokenized'))

# Now construct the full path to the file
file_path = os.path.join(parent_directory, 'tokenized_climateskeptics_sub_comments.pickle')

# Try to open the file using the full path
with open(file_path, 'rb') as f:
    token_ck_sub_com = pickle.load(f)

token_ck_sub_com.head()

,id,subreddit,body,author,score,gilded,created_utc,parent_id,link_id,retrieved_on,controversiality,is_submitter,tokenized_body_sents,tokenized_body_words,tokenized_body_words_norm
471,iqkdvfz,climateskeptics,"Yea, exactly the media blame every storm on cl...",dragonfirebreather,3,0,1664582500,iqkd247,xsg7fv,1664960560,0,True,"[yea, exactly the media blame every storm on c...","[yea, exactly, media, blame, storm, climate, c...","[yea, exactly, medium, blame, storm, climate, ..."
713,iqkebpl,climateskeptics,"We don’t deny climate change, we question the ...",the_stank__,9,0,1664582726,xsg17k,xsg17k,1664960546,0,False,"[we don’t deny climate change, we question the...","[deny, climate, change, question, validity, fa...","[deny, climate, change, question, validity, fa..."
924,iqkephh,climateskeptics,It’s not just solar irradiance. It’s also the ...,johnnyg883,1,0,1664582917,iqi8e8a,xrqls3,1664960535,0,False,"[it’s not just solar irradiance., it’s also th...","[solar, irradiance, solar, wind, cosmic, rays,...","[solar, irradiance, solar, wind, cosmic, ray, ..."
1082,iqkf0a3,climateskeptics,I’m with you. I’ll wait to see how cycle 25 pl...,johnnyg883,2,0,1664583065,iqig4fr,xrqls3,1664960526,0,False,"[i’m with you., i’ll wait to see how cycle 25 ...","[wait, cycle, plays, threw, crystal, ball, def...","[wait, cycle, play, throw, crystal, ball, defe..."
1323,iqkfghm,climateskeptics,That’s the most ridiculous comment I’ve seen s...,alexfromogish,1,0,1664583293,iqivabo,xrentr,1664960511,0,False,[that’s the most ridiculous comment i’ve seen ...,"[ridiculous, comment, seen, started, watching,...","[ridiculous, comment, see, start, watch, andth..."


In [9]:
# read in example dataset to help you understand what to do

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Import Dataset
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
#print(df.target_names.unique())
df.head()


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


In [15]:
# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]


In [17]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [19]:
print(data_words[0])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [22]:
# casting normalized words to a list to use going forward....?
token_ck_sub_com_list = token_ck_sub_com.tokenized_body_words_norm.values.tolist()

token_ck_sub_com_list[0]

['yea',
 'exactly',
 'medium',
 'blame',
 'storm',
 'climate',
 'change',
 'florida',
 'pirate',
 'caribbean',
 'territory',
 'hurricane',
 'normal',
 'high',
 'water',
 'temperature',
 'ask',
 'jack',
 'sparrow',
 'amp',
 'tell',
 'normal']

In [23]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(token_ck_sub_com_list, min_count=5, threshold=100) # JY: HAVE NOT YET THOUGHT ABOUT THESE NUMBERS
trigram = gensim.models.Phrases(bigram[token_ck_sub_com_list], threshold=100)  # JY: HAVE NOT YET THOUGHT ABOUT THESE NUMBERS 

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [24]:
# See trigram example
print(trigram_mod[bigram_mod[token_ck_sub_com_list[0]]])

['yea', 'exactly', 'medium', 'blame', 'storm', 'climate', 'change', 'florida', 'pirate', 'caribbean', 'territory', 'hurricane', 'normal', 'high', 'water', 'temperature', 'ask', 'jack', 'sparrow', 'amp', 'tell', 'normal']


In [27]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [30]:
# Form Bigrams
data_words_bigrams = make_bigrams(token_ck_sub_com_list)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
#nlp = spacy.load("en_core_web_sm")
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


In [31]:
print(data_lemmatized[0])

['exactly', 'medium', 'blame', 'storm', 'climate', 'change', 'territory', 'hurricane', 'normal', 'high', 'water', 'temperature', 'ask', 'tell', 'normal']


In [32]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1)]]


In [33]:
id2word[0]

'ask'

In [34]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('ask', 1),
  ('blame', 1),
  ('change', 1),
  ('climate', 1),
  ('exactly', 1),
  ('high', 1),
  ('hurricane', 1),
  ('medium', 1),
  ('normal', 2),
  ('storm', 1),
  ('tell', 1),
  ('temperature', 1),
  ('territory', 1),
  ('water', 1)]]

In [35]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [36]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.093*"uncertainty" + 0.074*"try" + 0.057*"problem" + 0.056*"scientific" + '
  '0.051*"issue" + 0.040*"information" + 0.039*"great" + 0.037*"hot" + '
  '0.036*"provide" + 0.036*"hurricane"'),
 (1,
  '0.246*"reduce" + 0.065*"politician" + 0.044*"prevent" + 0.041*"destroy" + '
  '0.037*"choice" + 0.030*"bill" + 0.019*"print" + 0.017*"offset" + '
  '0.000*"heat" + 0.000*"radiation"'),
 (2,
  '0.207*"know" + 0.107*"actually" + 0.086*"fact" + 0.060*"prediction" + '
  '0.051*"word" + 0.047*"prove" + 0.040*"write" + 0.031*"well" + 0.031*"fail" '
  '+ 0.030*"mention"'),
 (3,
  '0.168*"say" + 0.153*"science" + 0.077*"different" + 0.062*"true" + '
  '0.058*"start" + 0.054*"comment" + 0.052*"cold" + 0.046*"call" + '
  '0.042*"little" + 0.033*"man"'),
 (4,
  '0.103*"article" + 0.073*"read" + 0.065*"wrong" + 0.064*"day" + '
  '0.053*"impact" + 0.044*"old" + 0.038*"account" + 0.038*"live" + '
  '0.033*"free" + 0.032*"away"'),
 (5,
  '0.143*"include" + 0.086*"area" + 0.074*"post" + 0.069*"ref

In [37]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -17.05596264208413

Coherence Score:  0.3423718862169061


In [38]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

AttributeError: module 'pyLDAvis' has no attribute 'gensim'